<a href="https://colab.research.google.com/github/theKirill/GensimSample/blob/master/Practice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import gensim
import nltk
from nltk.corpus import stopwords
from gensim.models import Word2Vec
import pandas as pd
import numpy as np
import keras
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Flatten, Embedding, Dropout, Conv1D, GlobalMaxPooling1D, Activation
from keras.models import Model
import sklearn
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical

Using TensorFlow backend.


In [0]:
from keras.callbacks import ModelCheckpoint

In [0]:
from nltk.tokenize import RegexpTokenizer
from gensim.utils import simple_preprocess
import os

In [0]:
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [0]:
file_csv = pd.read_csv('drive/My Drive/ForGensim/train.csv')

In [0]:
file_csv

,text,intents
0,Add another song to the Cita RomГЎntica playli...,AddToPlaylist
1,add clem burke in my playlist Pre-Party R&B Jams,AddToPlaylist
2,Add Live from Aragon Ballroom to Trapeo,AddToPlaylist
3,add Unite and Win to my night out,AddToPlaylist
4,Add track to my Digster Future Hits,AddToPlaylist
...,...,...
11369,Is Across the Line playing at the closest movi...,SearchScreeningEvent
11370,Which animated movies are playing in the neigh...,SearchScreeningEvent
11371,Where is They Always Return at Dawn playing,SearchScreeningEvent
11372,What is the movie schedule in the neighborhood,SearchScreeningEvent


In [0]:
all_texts = file_csv['text']
all_intents = file_csv['intents']

In [0]:
file_csv = file_csv.dropna()
file_csv['text'].apply(lambda x: len(x.split(' '))).sum()

105347

In [0]:
file_csv['text']

0        Add another song to the Cita RomГЎntica playli...
1         add clem burke in my playlist Pre-Party R&B Jams
2                  Add Live from Aragon Ballroom to Trapeo
3                        add Unite and Win to my night out
4                      Add track to my Digster Future Hits
                               ...                        
11369    Is Across the Line playing at the closest movi...
11370    Which animated movies are playing in the neigh...
11371          Where is They Always Return at Dawn playing
11372       What is the movie schedule in the neighborhood
11373    Tell me when Howling II: Your Sister Is a Were...
Name: text, Length: 11374, dtype: object

In [0]:
unique_intents = file_csv.intents.unique()

In [0]:
unique_intents

array(['AddToPlaylist', 'BookRestaurant', 'GetWeather', 'PlayMusic',
       'SearchScreeningEvent'], dtype=object)

In [0]:
COUNT_CLASSES = intents.shape[0]


Удаление стоп-слов и пунктуации

In [0]:
sentences = np.array(all_texts.apply(lambda x : remove_stopwords(x)))

In [0]:
sentences

array([list(['add', 'another', 'song', 'cita', 'romгўntica', 'playlist']),
       list(['add', 'clem', 'burke', 'playlist', 'pre', 'party', 'r', 'b', 'jams']),
       list(['add', 'live', 'aragon', 'ballroom', 'trapeo']), ...,
       list(['always', 'return', 'dawn', 'playing']),
       list(['movie', 'schedule', 'neighborhood']),
       list(['tell', 'howling', 'ii', 'sister', 'werewolf', 'playing'])],
      dtype=object)

In [0]:
tokens = []

for sentence in sentences:
  tokens.append(sentence)

In [0]:
tokens[:5]

[['add', 'another', 'song', 'cita', 'romгўntica', 'playlist'],
 ['add', 'clem', 'burke', 'playlist', 'pre', 'party', 'r', 'b', 'jams'],
 ['add', 'live', 'aragon', 'ballroom', 'trapeo'],
 ['add', 'unite', 'win', 'night'],
 ['add', 'track', 'digster', 'future', 'hits']]

In [0]:
def remove_stopwords(text):
  regex_tokenizer = RegexpTokenizer(r'\w+')#не удаляет он цифры, надо юзать то, что я скинул!!!!!!!!!!!!! в словаре сраные числа остаются
  words = regex_tokenizer.tokenize(text.lower())
  stop_words = set(stopwords.words("english"))
  without_stop_words = [w for w in words if w not in stop_words]
  return without_stop_words

Работа с Word2Vec

In [0]:
VECTOR_SIZE = 200
WINDOW_SIZE = 10
MIN_COUNT = 3
ITER_COUNT = 10
WORKERS = 4

In [0]:
word2vec_model = Word2Vec(tokens, size = VECTOR_SIZE, window = WINDOW_SIZE, min_count = MIN_COUNT, workers = WORKERS, iter = ITER_COUNT)

In [0]:
#word2vec_model.train(tokens, total_examples = len(tokens), epochs = 500)

(22181828, 34511000)

In [0]:
word2vec_model['song']

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """Entry point for launching an IPython kernel.


array([ 0.25003234, -0.20251924, -0.21170434, -0.13942657,  0.06396154,
       -0.26132283,  0.15845542, -0.15610462,  0.12382077, -0.05135707,
        0.02039409, -0.38588202,  0.2072171 ,  0.5693021 ,  0.17661467,
       -0.17487907,  0.35937846,  0.04015371,  0.1262318 ,  0.00722068,
       -0.19969496,  0.52624655, -0.18651313, -0.54541147, -0.11531484,
       -0.04841517,  0.05331894, -0.14709659,  0.16798028, -0.06984915,
       -0.15223919, -0.00602355,  0.12938628,  0.11924691, -0.3358893 ,
       -0.06178682,  0.00385641,  0.14839341,  0.44979957, -0.23583919,
       -0.2102662 ,  0.25572878,  0.5060659 , -0.5088358 , -0.31146052,
        0.01500768,  0.02156699, -0.14977387, -0.6918172 , -0.14071584,
        0.10693123, -0.03573015, -0.0411291 , -0.23477852,  0.42286035,
       -0.00118758, -0.44126508,  0.3803695 , -0.53711647,  0.42044613,
       -0.2334012 , -0.19497752, -0.32737374,  0.17443003, -0.50338954,
        0.08378828,  0.2475111 , -0.2711518 , -0.3745477 ,  0.72

In [0]:
word2vec_model.most_similar('song')

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('track', 0.9996548295021057),
 ('artist', 0.9978812336921692),
 ('album', 0.9966175556182861),
 ('onto', 0.9942469000816345),
 ('playlist', 0.9930137991905212),
 ('put', 0.9926778078079224),
 ('tune', 0.9924858808517456),
 ('play', 0.9923551082611084),
 ('rock', 0.9914529323577881),
 ('add', 0.9908427000045776)]

In [0]:
word2vec_model.most_similar(positive=['song'], negative=['artist'])

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('ines', 0.29442721605300903),
 ('acгєstico', 0.13214555382728577),
 ('list', 0.1088150218129158),
 ('play', 0.09313375502824783),
 ('b', 0.0658533126115799),
 ('playing', 0.05466093868017197),
 ('kaori', 0.0470612570643425),
 ('music', 0.04589739441871643),
 ('theatres', 0.038758739829063416),
 ('movie', 0.03448214754462242)]

In [0]:
word2vec_model.save('drive/My Drive/ForGensim/Word2VecModel')
word2vec_model.wv.save_word2vec_format('drive/My Drive/ForGensim/Word2VecModel.txt', binary=False)
word2vec_model.wv.save_word2vec_format('drive/My Drive/ForGensim/Word2VecModel.bin', binary=True)

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:402: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


Работа с keras

In [0]:
word_vectors = word2vec_model.wv

In [0]:
word_vectors.vocab

{'add': <gensim.models.keyedvectors.Vocab at 0x7fe48eb2de48>,
 'another': <gensim.models.keyedvectors.Vocab at 0x7fe48eb2d5f8>,
 'song': <gensim.models.keyedvectors.Vocab at 0x7fe48eb2d048>,
 'romгўntica': <gensim.models.keyedvectors.Vocab at 0x7fe48eb2ddd8>,
 'playlist': <gensim.models.keyedvectors.Vocab at 0x7fe48eb2da90>,
 'clem': <gensim.models.keyedvectors.Vocab at 0x7fe48eb2d358>,
 'burke': <gensim.models.keyedvectors.Vocab at 0x7fe48eb2dc50>,
 'pre': <gensim.models.keyedvectors.Vocab at 0x7fe48eb2d0b8>,
 'party': <gensim.models.keyedvectors.Vocab at 0x7fe48eb2d2e8>,
 'r': <gensim.models.keyedvectors.Vocab at 0x7fe48eb2dcf8>,
 'b': <gensim.models.keyedvectors.Vocab at 0x7fe48eb2d390>,
 'jams': <gensim.models.keyedvectors.Vocab at 0x7fe48eb2d1d0>,
 'live': <gensim.models.keyedvectors.Vocab at 0x7fe48eb2dda0>,
 'aragon': <gensim.models.keyedvectors.Vocab at 0x7fe48eb35048>,
 'trapeo': <gensim.models.keyedvectors.Vocab at 0x7fe48eb35080>,
 'night': <gensim.models.keyedvectors.Vocab 

In [0]:
print("Number of word vectors: {}".format(len(word_vectors.vocab)))

Number of word vectors: 2584


In [0]:
texts = sentences

In [0]:
texts

array([list(['add', 'another', 'song', 'cita', 'romгўntica', 'playlist']),
       list(['add', 'clem', 'burke', 'playlist', 'pre', 'party', 'r', 'b', 'jams']),
       list(['add', 'live', 'aragon', 'ballroom', 'trapeo']), ...,
       list(['always', 'return', 'dawn', 'playing']),
       list(['movie', 'schedule', 'neighborhood']),
       list(['tell', 'howling', 'ii', 'sister', 'werewolf', 'playing'])],
      dtype=object)

In [0]:
kek = texts[0]
kek

['add', 'another', 'song', 'cita', 'romгўntica', 'playlist']

In [0]:
#vocab_size = len(word_vectors.vocab)
#tokenizer = Tokenizer(vocab_size)
tokenizer = Tokenizer()
tokenizer.fit_on_texts(texts)
text = tokenizer.texts_to_sequences(texts)
text = pad_sequences(text, maxlen = 10)

In [0]:
X_train, X_test, y_train, y_test = train_test_split(text, all_intents, test_size = 0.2, stratify = all_intents)

In [0]:
y_train = transform_to_categorical(y_train, unique_intents.tolist())

In [0]:
y_test =  transform_to_categorical(y_test, unique_intents.tolist())

In [0]:
keras_model = get_model(word2vec_model, COUNT_CLASSES)
keras_model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, None, 200)         516800    
_________________________________________________________________
dropout_5 (Dropout)          (None, None, 200)         0         
_________________________________________________________________
conv1d_3 (Conv1D)            (None, None, 50)          30050     
_________________________________________________________________
global_max_pooling1d_3 (Glob (None, 50)                0         
_________________________________________________________________
dense_5 (Dense)              (None, 250)               12750     
_________________________________________________________________
dropout_6 (Dropout)          (None, 250)               0         
_________________________________________________________________
activation_3 (Activation)    (None, 250)              

In [0]:
BATCH = 32
EPOCHS_COUNT = 10

In [0]:
keras_model.fit(X_train, y_train, batch_size = BATCH, epochs = EPOCHS_COUNT, validation_data = (X_test, y_test),
             callbacks=[ModelCheckpoint('drive/My Drive/ForGensim/keras_model.h5', save_best_only = True)])

Train on 9099 samples, validate on 2275 samples
Epoch 1/10
9099/9099 [==============================] - 3s 311us/step - loss: 0.5235 - acc: 0.8045 - val_loss: 0.0600 - val_acc: 0.9859
Epoch 2/10
9099/9099 [==============================] - 2s 247us/step - loss: 0.0601 - acc: 0.9837 - val_loss: 0.0398 - val_acc: 0.9899
Epoch 3/10
9099/9099 [==============================] - 2s 250us/step - loss: 0.0335 - acc: 0.9910 - val_loss: 0.0353 - val_acc: 0.9908
Epoch 4/10
9099/9099 [==============================] - 2s 253us/step - loss: 0.0181 - acc: 0.9956 - val_loss: 0.0323 - val_acc: 0.9925
Epoch 5/10
9099/9099 [==============================] - 2s 247us/step - loss: 0.0112 - acc: 0.9971 - val_loss: 0.0286 - val_acc: 0.9921
Epoch 6/10
9099/9099 [==============================] - 2s 261us/step - loss: 0.0080 - acc: 0.9975 - val_loss: 0.0356 - val_acc: 0.9916
Epoch 7/10
9099/9099 [==============================] - 2s 251us/step - loss: 0.0053 - acc: 0.9986 - val_loss: 0.0337 - val_acc: 0.9921


In [0]:
X_predict = []
X_predict.append([  0,    0,    0,    9,    2, 2716,    3,  179, 2717,   85])# не знаем, как строку преобразовать в вектор чисел
X_predict = np.array(X_predict)

In [0]:
prediction = keras_model.predict_classes(X_predict)

In [0]:
answers = transform_from_categorical(prediction, unique_intents.tolist())
answers

['AddToPlaylist']

In [0]:
# Создание модели
def get_model(word2vec_model, COUNT_CLASSES):
    model = Sequential() 
    model.add(word2vec_model.wv.get_keras_embedding(train_embeddings = True))
    
    model.add(Dropout(0.2))

    model.add(Conv1D(50,
                 3,
                 padding = 'valid',
                 activation = 'relu',
                 strides = 1))
    model.add(GlobalMaxPooling1D())

    model.add(Dense(250))
    model.add(Dropout(0.2))
    model.add(Activation('relu'))

    model.add(Dense(COUNT_CLASSES, activation = 'softmax'))

    model.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
  
    return model

In [0]:
def transform_to_categorical(intents_for_each, unique_intents):
  intents_to_digit = []

  for intent in intents_for_each:
    intents_to_digit.append(unique_intents.index(intent))

  return to_categorical(intents_to_digit, 7)

In [0]:
def transform_from_categorical(prediction, intents):
  answers = []

  for i in range(prediction.shape[0]):
    answers.append(intents[prediction[i]])

  return answers

**Дообучение модели**

In [0]:
new_file_csv = pd.read_csv('drive/My Drive/ForGensim/train_new.csv')

In [0]:
all_new_texts = new_file_csv['text']
all_new_intents = new_file_csv['intents']

In [0]:
unique_new_intents = new_file_csv.intents.unique()

In [0]:
unique_new_intents

array(['RateBook', 'SearchCreativeWork'], dtype=object)

In [0]:
from keras.models import save_model, load_model

In [0]:
keras_model = load_model('drive/My Drive/ForGensim/keras_model.h5')

In [0]:
word2vec_model = Word2Vec.load("drive/My Drive/ForGensim/Word2VecModel")

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:402: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [0]:
sentences = np.array(all_new_texts.apply(lambda x : remove_stopwords(x)))

In [0]:
sentences

array([list(['rate', 'lotus', 'storm', 'zero', '6']),
       list(['rate', 'fall', 'artist', '5', 'stars']),
       list(['rate', 'current', 'novel', 'one', 'points']), ...,
       list(['please', 'get', 'follow', 'video', 'game']),
       list(['please', 'look', 'show', 'statue', 'james', 'outram']),
       list(['find', 'far', 'tv', 'series'])], dtype=object)

In [0]:
texts = sentences

In [0]:
tokens = []

for sentence in sentences:
  tokens.append(sentence)

In [0]:
#дообучаем модель word2vec
word2vec_model.build_vocab(tokens, update = True)
word2vec_model.train(tokens, total_examples = len(tokens), epochs = word2vec_model.iter)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: DeprecationWarning: Call to deprecated `iter` (Attribute will be removed in 4.0.0, use self.epochs instead).
  


(103452, 236420)

In [0]:
token = Tokenizer()
token.fit_on_texts(texts)
text = token.texts_to_sequences(texts)
text = pad_sequences(text, maxlen=10)

In [0]:
X_train_new, X_test_new, y_train_new, y_test_new = train_test_split(text, all_new_intents, test_size=0.2, stratify=all_new_intents)

In [0]:
all_unique_intents=unique_intents.tolist()

In [0]:
for i in unique_new_intents:
  all_unique_intents.append(i)

In [207]:
all_unique_intents

['AddToPlaylist',
 'BookRestaurant',
 'GetWeather',
 'PlayMusic',
 'SearchScreeningEvent',
 'RateBook',
 'SearchCreativeWork']

In [0]:
y_train_new = transform_to_categorical(y_train_new, all_unique_intents)

In [0]:
y_test_new =  transform_to_categorical(y_test_new, all_unique_intents)

In [0]:
keras_model.pop()

In [0]:
keras_model.add(Dense(7, activation = 'softmax'))

In [0]:
keras_model.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])

In [265]:
keras_model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, None, 200)         516800    
_________________________________________________________________
dropout_5 (Dropout)          (None, None, 200)         0         
_________________________________________________________________
conv1d_3 (Conv1D)            (None, None, 50)          30050     
_________________________________________________________________
global_max_pooling1d_3 (Glob (None, 50)                0         
_________________________________________________________________
dense_5 (Dense)              (None, 250)               12750     
_________________________________________________________________
dropout_6 (Dropout)          (None, 250)               0         
_________________________________________________________________
activation_3 (Activation)    (None, 250)              

In [268]:
keras_model.fit(X_train_new, y_train_new, batch_size=BATCH, epochs=EPOCHS_COUNT, validation_data=(X_test_new, y_test_new))

Train on 3608 samples, validate on 902 samples
Epoch 1/10
3608/3608 [==============================] - 2s 610us/step - loss: 0.2963 - acc: 0.8872 - val_loss: 0.0568 - val_acc: 0.9800
Epoch 2/10
3608/3608 [==============================] - 1s 270us/step - loss: 0.0382 - acc: 0.9881 - val_loss: 0.0254 - val_acc: 0.9922
Epoch 3/10
3608/3608 [==============================] - 1s 292us/step - loss: 0.0144 - acc: 0.9961 - val_loss: 0.0152 - val_acc: 0.9933
Epoch 4/10
3608/3608 [==============================] - 1s 269us/step - loss: 0.0089 - acc: 0.9981 - val_loss: 0.0127 - val_acc: 0.9956
Epoch 5/10
3608/3608 [==============================] - 1s 276us/step - loss: 0.0021 - acc: 0.9997 - val_loss: 0.0117 - val_acc: 0.9956
Epoch 6/10
3608/3608 [==============================] - 1s 272us/step - loss: 9.9655e-04 - acc: 1.0000 - val_loss: 0.0098 - val_acc: 0.9956
Epoch 7/10
3608/3608 [==============================] - 1s 272us/step - loss: 7.7801e-04 - acc: 0.9997 - val_loss: 0.0163 - val_acc: 

In [0]:
X_predict = []
X_predict.append([  0,    0,    0,    9,    2, 2716,    3,  179, 2717,   85])# не знаем, как строку преобразовать в вектор чисел
X_predict = np.array(X_predict)

In [0]:
prediction = keras_model.predict_classes(X_predict)

In [273]:
answers = transform_from_categorical(prediction, all_unique_intents)
answers

['SearchCreativeWork']